# Setup

In [ ]:
!pip install fiftyone -q

# Download dataset

**You can obtain your link (Roboflow key) on the download page [here](https://universe.roboflow.com/brad-dwyer/aquarium-combined/dataset/5/download) (requires sign-up)**

In [ ]:
# %env YOUR_ROBOFLOW_KEY=

In [ ]:
import os

if "YOUR_ROBOFLOW_KEY" not in os.environ:
  raise RuntimeError("Set YOUR_ROBOFLOW_KEY environmental variable above!")

In [ ]:
!curl -L "https://universe.roboflow.com/ds/L0q2Ht5q1V?key=$YOUR_ROBOFLOW_KEY" > roboflow.zip;
!unzip -o -q roboflow.zip -d data; rm roboflow.zip

**Insert path to dataset root directory**

In [ ]:
# NOTE - Update with your path if not running in Google Colab
!sed -i "1i path: /content/data" data/data.yaml

**Fix mislabeled annotation**

In [ ]:
%env mislabeled_file=data/train/labels/IMG_8590_MOV-3_jpg.rf.e215fd21f9f69e42089d252c40cc2608.txt
!awk '{print "1", $2, $3, $4, $5}' $mislabeled_file > tmp.txt && mv tmp.txt $mislabeled_file

# Download JSON with model predictions

In [ ]:
!wget -q https://github.com/abojda/aquarium-object-detection/raw/main/results/yolov5/evaluation/test_s_obj0.1_300/predictions_fixed.json

# Run FiftyOne

In [ ]:
import fiftyone as fo

dataset = fo.Dataset.from_dir(
    name='Aquarium Combined',
    dataset_type=fo.types.COCODetectionDataset,
    data_path='data/test',
    labels_path='data/test/_annotations.coco.json',
    label_field='ground_truth',
)

pred_dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path='data/test',
    labels_path='predictions_fixed.json',
    label_field='model',
)

dataset.merge_samples(pred_dataset)

session = fo.launch_app(dataset)